In [1]:
!conda install beautifulsoup4
!conda install lxml
!conda install requests
print('Packages installed successfully')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.1       |           py36_0         153 KB

The following packages will be UPDATED:

    beautifulsoup4: 4.7.1-py36_1 --> 4.8.1-py36_0


beautifulsoup4-4.8.1 | 153 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.4.2                 |   py36hefd8a0e_0         1.6 MB

The following packages will be UPDATED:


In [24]:
from bs4 import BeautifulSoup
import requests

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes  
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes  
import folium # map rendering library

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
     


print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

In [4]:
table = soup.find('table')
table_rows = table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    l.append(row)
df = pd.DataFrame(l, columns=["Postcode", "Borough", "Neighborhood"])

In [5]:
df1=df.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join).reset_index()

In [6]:
df2 =df1.drop(df1[df1.Borough == 'Not assigned'].index).reset_index()
del df2['index']

In [7]:
df2.loc[df2['Neighborhood'] 
        == 'Not assigned', 'Neighborhood'] = df2['Borough']
df2.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
#Checking
df_chk = df2.loc[df2['Postcode']=="M7A"]
df_chk

,Postcode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [9]:
df2.shape


(103, 3)

In [10]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [11]:
Geospatial_df = pd.read_csv('Geospatial_data.csv')
Geospatial_df.rename(columns={'Postal Code':'Postcode'}, inplace=True)
Geospatial_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
Geospatial_df.shape

(103, 3)

In [13]:
df3 = pd.merge(df2, Geospatial_df, on='Postcode')
df3.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
df3.shape

(103, 5)

In [15]:
df_toronto = df3[df3['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [16]:
df_toronto['Borough'].unique()

array(['East Toronto', 'Central Toronto', 'Downtown Toronto',
       'West Toronto'], dtype=object)

In [17]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [25]:
CLIENT_ID = 'ETETNADXSHKVPM2HSVKKV5YVCBLDT1LZ1RS1QUW2T2MMBX3K' # your Foursquare ID
CLIENT_SECRET = '2TFUPJKJ005UZRFI1OMTLPHZ0GCM5WFHXDQQZWICD13M0YSL' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ETETNADXSHKVPM2HSVKKV5YVCBLDT1LZ1RS1QUW2T2MMBX3K
CLIENT_SECRET:2TFUPJKJ005UZRFI1OMTLPHZ0GCM5WFHXDQQZWICD13M0YSL


In [30]:
LIMIT =500

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [32]:
print(toronto_venues.shape)
toronto_venues.head()

(3102, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
2,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
3,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
4,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub


In [33]:
feat_name_list1 = ['Bakery',
'Vegetarian / Vegan Restaurant',
'Ice Cream Shop',
'Gastropub',
'Bagel Shop',
'Breakfast Spot',
'Coffee Shop',
'French Restaurant',
'Pub',
'Mexican Restaurant',
'Bar',
'Tea Room',
'Japanese Restaurant',
'Juice Bar',
'Caribbean Restaurant',
'Cupcake Shop',
'Diner',
'Ramen Restaurant',
'Indian Restaurant',
'Chocolate Shop',
'Sandwich Place',
'Thai Restaurant',
'Burger Joint',
'Café',
'Asian Restaurant',
'Beer Store',
'Pizza Place',
'Greek Restaurant',
'Restaurant',
'Italian Restaurant',
'Brewery',
'Dessert Shop',
'Bubble Tea Shop',
'BBQ Joint',
'Tapas Restaurant',
'Donut Shop',
'American Restaurant',
'Cuban Restaurant',
'Turkish Restaurant',
'New American Restaurant',
'Sushi Restaurant',
'Portuguese Restaurant',
'Fish & Chips Shop',
'Fast Food Restaurant',
'Falafel Restaurant',
'Chinese Restaurant',
'Pakistani Restaurant','Burrito Place',
'Steakhouse',
'Snack Place',
'Indian Chinese Restaurant',
'Middle Eastern Restaurant',
'Taco Place',
'Comfort Food Restaurant',
'Hotel',
'Latin American Restaurant',
'Dive Bar',
'Fried Chicken Joint',
'Vietnamese Restaurant',
'Seafood Restaurant',
'Hotel Bar',
'Food & Drink Shop',
'Wine Bar',
'Deli / Bodega',
'Salad Place',
'Smoothie Shop',
'Candy Store',
'Buffet',
'Indonesian Restaurant',
'Wings Joint',
'Cantonese Restaurant',
'Modern European Restaurant',
'German Restaurant',
'Filipino Restaurant',
'Pie Shop',
'Taiwanese Restaurant',
'Piano Bar',
'Gay Bar',
'Ethiopian Restaurant',
'Sake Bar',
'Afghan Restaurant',
'Persian Restaurant',
'Mediterranean Restaurant',
'Food Truck',
'Beer Bar',
'Cocktail Bar',
'Bistro',
'Belgian Restaurant''Food Court',
'Noodle House',
'Brazilian Restaurant',
'Gluten-free Restaurant',
'Irish Pub',
'Jewish Restaurant',
'Korean Restaurant',
'Eastern European Restaurant',
'Dumpling Restaurant',
'Doner Restaurant',
'Arepa Restaurant',
'South American Restaurant',
'Malay Restaurant',
'Mac & Cheese Joint',
'Southern / Soul Food Restaurant',
'Soup Place',
'Whisky Bar',
'Tibetan Restaurant',
'Hawaiian Restaurant',
'Beach Bar',
'Cajun / Creole Restaurant',
'Food',
'Frozen Yogurt Shop']
                
feat_name_list2 = ['Indie Movie Theater',
'Jazz Club',
'Concert Hall',
'Dance Studio',
'Indie Theater',
'Movie Theater',
'Rock Club',
'Gaming Cafe',
'General Entertainment',
'Theater',
'Performing Arts Venue',
'Hobby Shop',
'Nightclub',
'Video Game Store',
'Video Store',
'Event Space',
'Design Studio',
'Art Gallery',
'Karaoke Bar',
'Art Museum',
'Museum',
'Exhibit',
'Opera House',
'History Museum',
'Music Venue',
'Pool Hall',
'Street Art',
'Comedy Club',
'Amphitheater',
'Arts & Entertainment',
]
feat_name_list3 = ['Baseball Field',
'Sporting Goods Shop',
'Track',
'Skating Rink',
'Tennis Court',
'Athletics & Sports',
'Basketball Stadium',
'Baseball Stadium',
'Stadium',
'Soccer Stadium',
]
feat_name_list4 = ['College Quad',
'College Gym',
'Library',
'University',
'Music School',
'School','College Arts Building',
'College Lab',
]
feat_name_list5 =['Yoga Studio',
'Gym',
'Climbing Gym',
'Gym / Fitness Center',
'Gym Pool',
'Pilates Studio',
'Martial Arts Dojo',
'Rock Climbing Spot',
]
feat_name_list6=['Nail Salon',
'Salon / Barbershop',
'Spa',
'Health & Beauty Service',
'Bridal Shop',
]
feat_name_list7=['Park',
'Beach',
'Scenic Lookout',
'Tree',
'Pool',
'Skate Park',
'Garden Center',
'Harbor / Marina',
'Garden',
'Playground',
'Farm',
'Historic Site',
'Monument / Landmark',
'Fountain',
'Lake',
'Aquarium',
'Castle',
'Sculpture Garden',
'Other Great Outdoors',
'Zoo',
'River',
]
feat_name_list8 = ['Toy / Game Store',
'Supermarket',
'Bookstore',
'Pharmacy',
'Bank',
'Grocery Store',
'Electronics Store',
'Pet Store',
'Clothing Store',
'Flower Shop',
's Store"',
'Jewelry Store',
'Shopping Mall',
'Cosmetics Shop',
'Health Food Store',
'Fruit & Vegetable Store',
'Liquor Store',
'Furniture / Home Store',
'Discount Store',
'Farmers Market',
'Comic Shop',
'Butcher',
'Smoke Shop',
'Fish Market',
'Stationery Store',
'Cheese Shop',
'Boutique',
'Antique Shop',
'Arts & Crafts Store',
'Convenience Store',
'Plaza',
'Gourmet Shop',
'Gift Shop',
'Sports Bar',
'Miscellaneous Shop',
's Store',
'Adult Boutique','Shoe Store',
'Animal Shelter',
'Department Store',
'Optical Shop',
'Tailor Shop',
'Souvlaki Shop',
'Lingerie Store',
'Music Store',
'Record Shop',
'Organic Grocery',
'Paper / Office Supplies Store',
'Other Repair Shop',
'Thrift / Vintage Store',
'Accessories Store',
'Flea Market',
'Hardware Store',
'Gas Station',
]
feat_name_list9 =['Bus Stop',
'Light Rail Station',
'Rental Car Location',
'Metro Station',
'Train Station',
'General Travel',
'Airport',
'Airport Lounge',
'Bus Line',
]
toronto_venues['Category'] ='Others'
toronto_venues.loc[toronto_venues['Venue Category'].isin(feat_name_list1),'Category']='Food and Dine'
toronto_venues.loc[toronto_venues['Venue Category'].isin(feat_name_list2),'Category']='Arts and Entertainment'
toronto_venues.loc[toronto_venues['Venue Category'].isin(feat_name_list3),'Category']='Athletics and Sports'
toronto_venues.loc[toronto_venues['Venue Category'].isin(feat_name_list4),'Category']='Educational'
toronto_venues.loc[toronto_venues['Venue Category'].isin(feat_name_list5),'Category']='Gym and Fitness Center'
toronto_venues.loc[toronto_venues['Venue Category'].isin(feat_name_list6),'Category']='Health and Beauty Service'
toronto_venues.loc[toronto_venues['Venue Category'].isin(feat_name_list7),'Category']='Recreation'
toronto_venues.loc[toronto_venues['Venue Category'].isin(feat_name_list8),'Category']='Stores and Utilities'
toronto_venues.loc[toronto_venues['Venue Category'].isin(feat_name_list9),'Category']='Transportation'

In [34]:
list(toronto_venues['Category'].unique())

['Others',
 'Food and Dine',
 'Arts and Entertainment',
 'Stores and Utilities',
 'Recreation',
 'Health and Beauty Service',
 'Athletics and Sports',
 'Gym and Fitness Center',
 'Transportation',
 'Educational']

In [35]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail,Others
1,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant,Food and Dine
2,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater,Arts and Entertainment
3,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery,Food and Dine
4,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub,Food and Dine


In [36]:
LIMIT =500
search_query ='Hospital'
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
               
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            search_query)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude']
    
    return(nearby_venues)

In [37]:
toronto_hospitals = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [38]:
toronto_hospitals['Category'] = 'Hospitals'
toronto_hospitals.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Category
0,The Beaches,43.676357,-79.293031,Beaches Animal Hospital,43.673449,-79.283777,Hospitals
1,The Beaches,43.676357,-79.293031,Boardwalk Animal Hospital - In The Beaches,43.671459,-79.293374,Hospitals
2,The Beaches,43.676357,-79.293031,Boardwalk Animal Hospital,43.670814,-79.296732,Hospitals
3,The Beaches,43.676357,-79.293031,Beaches Animal Hospital,43.673454,-79.283776,Hospitals
4,The Beaches,43.676357,-79.293031,Kingston Road Animalhosp,43.680760,-79.284721,Hospitals


In [39]:
list(toronto_hospitals['Venue'].unique())

['Beaches Animal Hospital',
 'Boardwalk Animal Hospital - In The Beaches',
 'Boardwalk Animal Hospital',
 'Kingston Road Animalhosp',
 'Blue Cross Animal Hospital',
 'Danforth Animal Hospital',
 'Riverdale Animal Hospital',
 'Banks Animal Hospital',
 'Ashbridges Bay Animal Hospital',
 'Blacks Veterinary Hospital',
 'Toronto Veterinary Hospital',
 'Bay Dog Hospital',
 'Leslieville Animal Hospital',
 'Emergency Room: Sunnybrook Hospital',
 'Sunnybrook Hospital',
 'Sunnybrook Hospital Critical Care Unit',
 'Lawrence Park Animal Hospital',
 'Neurology Unit: Sunnybrook Hospital',
 'K Wing: Sunnybrook Hospital',
 'Sunnybrook Health Sciences Centre',
 'Second Cup',
 'M-wing: Sunnybrook',
 'FUS Lab: Sunnybrook',
 'D Wing: Sunnybrook',
 'T-wing: Sunnybrook',
 'Mt. Pleasant-Davisville Veterinary Hospital',
 'Sunnybrook Hospital Kidney Care Clinic',
 'Davisville Park Animal Hospital',
 'Yonge Street Animal Hospital',
 'Tim Hortons',
 'McGilvray Veterinary Hospital',
 'Usher Animal Hospital',
 'VI

In [40]:
hosp_list = [ 'Sunnybrook Health Sciences Centre',
 'Toronto Grace Health Centre',
 "410 Health Centre (St. Michael's Hospital)",
 'The Hospital for Sick Children (SickKids)',
 'Toronto General Hospital',
 "Women's College Hospital",
 'Mount Sinai Hospital',
 'Princess Margaret Hospital Foundation',
 'St Michael Hospital',
 'Rouge Valley Hospital',
 'Council of Academic Hospitals of Ontario',
 'Mr Sub - Toronto Western Hospital',
 'Toronto Western Hospital',
 "St. Joseph's Health Centre"]
toronto_hosp = toronto_hospitals[toronto_hospitals['Venue'].isin(hosp_list)].reset_index()
toronto_hosp.head()

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Category
0,21,Lawrence Park,43.728020,-79.388790,Sunnybrook Health Sciences Centre,43.721505,-79.376210,Hospitals
1,42,Rosedale,43.679563,-79.377529,Toronto Grace Health Centre,43.670640,-79.383263,Hospitals
2,43,"Cabbagetown,St. James Town",43.667967,-79.367675,410 Health Centre (St. Michael's Hospital),43.664832,-79.374050,Hospitals
3,45,Church and Wellesley,43.665860,-79.383160,The Hospital for Sick Children (SickKids),43.657499,-79.386512,Hospitals
4,46,Church and Wellesley,43.665860,-79.383160,Toronto General Hospital,43.658762,-79.388292,Hospitals


In [41]:
LIMIT =100
search_query ='School'
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
               
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            search_query)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude']
    return(nearby_venues)

In [42]:
toronto_schools = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [43]:
toronto_schools['Category'] = 'Educational'
toronto_schools.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Category
0,The Beaches,43.676357,-79.293031,St.John Catholic School,43.680676,-79.294542,Educational
1,The Beaches,43.676357,-79.293031,Beach Swim School,43.682231,-79.289350,Educational
2,The Beaches,43.676357,-79.293031,St. Denis Catholic School,43.672881,-79.290056,Educational
3,The Beaches,43.676357,-79.293031,Balmy Beach School,43.676199,-79.290134,Educational
4,The Beaches,43.676357,-79.293031,St. John's School Playground,43.680739,-79.296430,Educational


In [44]:
toronto_new = pd.concat([toronto_venues,toronto_hosp,toronto_schools],axis =0)
toronto_new.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude,index
0,Others,The Beaches,43.676357,-79.293031,Glen Manor Ravine,Trail,43.676821,-79.293942,NaN
1,Food and Dine,The Beaches,43.676357,-79.293031,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331,NaN
2,Arts and Entertainment,The Beaches,43.676357,-79.293031,The Fox Theatre,Indie Movie Theater,43.672801,-79.287272,NaN
3,Food and Dine,The Beaches,43.676357,-79.293031,Beaches Bake Shop,Bakery,43.680363,-79.289692,NaN
4,Food and Dine,The Beaches,43.676357,-79.293031,The Beech Tree,Gastropub,43.680493,-79.288846,NaN


In [45]:
toronto_sorted = toronto_new.sort_values(by = 'Neighborhood').reset_index()
toronto_sorted = toronto_sorted.drop(['index'],axis =1)
toronto_sorted.head()

,level_0,Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
0,1492,Food and Dine,"Adelaide,King,Richmond",43.650571,-79.384568,The Fairmont Royal York,Hotel,43.645449,-79.381508
1,1465,Food and Dine,"Adelaide,King,Richmond",43.650571,-79.384568,Kupfert & Kim (First Canadian Place),Gluten-free Restaurant,43.648547,-79.381624
2,1466,Food and Dine,"Adelaide,King,Richmond",43.650571,-79.384568,Starbucks,Coffee Shop,43.646731,-79.383951
3,1467,Gym and Fitness Center,"Adelaide,King,Richmond",43.650571,-79.384568,GoodLife Fitness Toronto 137 Yonge Street,Gym,43.651242,-79.378068
4,1468,Food and Dine,"Adelaide,King,Richmond",43.650571,-79.384568,HotBlack Coffee,Coffee Shop,43.650364,-79.388669


In [46]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_sorted[['Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_sorted['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Arts and Entertainment,Athletics and Sports,Educational,Food and Dine,Gym and Fitness Center,Health and Beauty Service,Hospitals,Others,Recreation,Stores and Utilities,Transportation
0,"Adelaide,King,Richmond",0,0,0,1,0,0,0,0,0,0,0
1,"Adelaide,King,Richmond",0,0,0,1,0,0,0,0,0,0,0
2,"Adelaide,King,Richmond",0,0,0,1,0,0,0,0,0,0,0
3,"Adelaide,King,Richmond",0,0,0,0,1,0,0,0,0,0,0
4,"Adelaide,King,Richmond",0,0,0,1,0,0,0,0,0,0,0


In [47]:
toronto_onehot_new=toronto_onehot.set_index('Neighborhood')
toronto_onehot_new.head()

,Arts and Entertainment,Athletics and Sports,Educational,Food and Dine,Gym and Fitness Center,Health and Beauty Service,Hospitals,Others,Recreation,Stores and Utilities,Transportation
Neighborhood,,,,,,,,,,,
"Adelaide,King,Richmond",0,0,0,1,0,0,0,0,0,0,0
"Adelaide,King,Richmond",0,0,0,1,0,0,0,0,0,0,0
"Adelaide,King,Richmond",0,0,0,1,0,0,0,0,0,0,0
"Adelaide,King,Richmond",0,0,0,0,1,0,0,0,0,0,0
"Adelaide,King,Richmond",0,0,0,1,0,0,0,0,0,0,0


In [48]:
toronto_onehot_new = toronto_onehot_new.groupby('Neighborhood').sum()
toronto_onehot_new.head()

,Arts and Entertainment,Athletics and Sports,Educational,Food and Dine,Gym and Fitness Center,Health and Beauty Service,Hospitals,Others,Recreation,Stores and Utilities,Transportation
Neighborhood,,,,,,,,,,,
"Adelaide,King,Richmond",11,0,51,62,4,2,6,4,1,14,1
Berczy Park,6,2,27,65,2,0,0,6,6,12,1
"Brockton,Exhibition Place,Parkdale Village",5,3,13,64,4,1,0,5,2,16,0
Business Reply Mail Processing Centre 969 Eastern,0,0,4,32,2,0,0,0,8,8,0
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",1,1,4,4,0,0,0,1,6,0,2


In [49]:
toronto_onehot_new.shape

(39, 11)

In [50]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_onehot_new)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 4, 1, 3, 1, 0, 0, 2, 0], dtype=int32)

In [51]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = toronto_onehot_new.columns
means_df.index=['C1','C2','C3','C4','C5']
means_df['Facilities']=means_df.sum(axis = 1)
means_df.sort_values(axis=0,by=['Facilities'],ascending =False)

,Arts and Entertainment,Athletics and Sports,Educational,Food and Dine,Gym and Fitness Center,Health and Beauty Service,Hospitals,Others,Recreation,Stores and Utilities,Transportation,Facilities
C1,6.909091,1.000000,49.818182,66.727273,2.909091,0.545455,4.636364,4.727273,3.272727,13.090909,4.545455e-01,154.090909
C3,5.444444,0.888889,23.777778,70.444444,3.000000,1.000000,0.000000,3.111111,3.111111,12.555556,1.111111e-01,123.444444
C5,3.750000,1.875000,13.625000,56.125000,2.500000,1.375000,0.250000,3.250000,6.500000,12.750000,1.125000e+00,103.125000
C2,1.166667,1.500000,15.666667,30.833333,2.500000,0.500000,0.166667,1.500000,5.666667,7.333333,-5.551115e-17,66.833333
C4,0.200000,0.800000,8.600000,6.800000,0.600000,0.200000,0.400000,1.000000,2.400000,3.400000,6.000000e-01,25.000000


In [52]:
#toronto_onehot_new = toronto_onehot_new.set_index('Neighborhood')
toronto_onehot_new['Cluster']=1+kmeans.labels_
toronto_onehot_new.head()

,Arts and Entertainment,Athletics and Sports,Educational,Food and Dine,Gym and Fitness Center,Health and Beauty Service,Hospitals,Others,Recreation,Stores and Utilities,Transportation,Cluster
Neighborhood,,,,,,,,,,,,
"Adelaide,King,Richmond",11,0,51,62,4,2,6,4,1,14,1,1
Berczy Park,6,2,27,65,2,0,0,6,6,12,1,3
"Brockton,Exhibition Place,Parkdale Village",5,3,13,64,4,1,0,5,2,16,0,5
Business Reply Mail Processing Centre 969 Eastern,0,0,4,32,2,0,0,0,8,8,0,2
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",1,1,4,4,0,0,0,1,6,0,2,4


In [53]:
toronto_final = df_toronto.set_index('Neighborhood').join(toronto_onehot_new)
toronto_final.head()

,Postcode,Borough,Latitude,Longitude,Arts and Entertainment,Athletics and Sports,Educational,Food and Dine,Gym and Fitness Center,Health and Beauty Service,Hospitals,Others,Recreation,Stores and Utilities,Transportation,Cluster
Neighborhood,,,,,,,,,,,,,,,,
The Beaches,M4E,East Toronto,43.676357,-79.293031,1,1,18,55,0,3,0,3,9,12,0,5
"The Danforth West,Riverdale",M4K,East Toronto,43.679557,-79.352188,2,0,16,70,2,4,0,4,3,15,0,3
"The Beaches West,India Bazaar",M4L,East Toronto,43.668999,-79.315572,4,1,7,49,3,1,0,2,11,10,1,5
Studio District,M4M,East Toronto,43.659526,-79.340923,1,0,12,71,5,0,0,5,2,16,0,3
Lawrence Park,M4N,Central Toronto,43.728020,-79.388790,0,0,11,3,1,0,1,1,1,1,0,4


In [54]:
toronto_final.reset_index(level=0, inplace=True)
toronto_final.head()

,Neighborhood,Postcode,Borough,Latitude,Longitude,Arts and Entertainment,Athletics and Sports,Educational,Food and Dine,Gym and Fitness Center,Health and Beauty Service,Hospitals,Others,Recreation,Stores and Utilities,Transportation,Cluster
0,The Beaches,M4E,East Toronto,43.676357,-79.293031,1,1,18,55,0,3,0,3,9,12,0,5
1,"The Danforth West,Riverdale",M4K,East Toronto,43.679557,-79.352188,2,0,16,70,2,4,0,4,3,15,0,3
2,"The Beaches West,India Bazaar",M4L,East Toronto,43.668999,-79.315572,4,1,7,49,3,1,0,2,11,10,1,5
3,Studio District,M4M,East Toronto,43.659526,-79.340923,1,0,12,71,5,0,0,5,2,16,0,3
4,Lawrence Park,M4N,Central Toronto,43.728020,-79.388790,0,0,11,3,1,0,1,1,1,1,0,4


In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_final['Latitude'], toronto_final['Longitude'], toronto_final['Neighborhood'], toronto_final['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [56]:
toronto_final[toronto_final['Cluster'] == 3]

,Neighborhood,Postcode,Borough,Latitude,Longitude,Arts and Entertainment,Athletics and Sports,Educational,Food and Dine,Gym and Fitness Center,Health and Beauty Service,Hospitals,Others,Recreation,Stores and Utilities,Transportation,Cluster
1,"The Danforth West,Riverdale",M4K,East Toronto,43.679557,-79.352188,2,0,16,70,2,4,0,4,3,15,0,3
3,Studio District,M4M,East Toronto,43.659526,-79.340923,1,0,12,71,5,0,0,5,2,16,0,3
5,Davisville North,M4P,Central Toronto,43.712751,-79.390197,3,3,23,73,5,2,0,1,1,12,0,3
7,Davisville,M4S,Central Toronto,43.704324,-79.388790,5,1,27,75,4,0,0,2,1,12,0,3
13,Harbourfront,M5A,Downtown Toronto,43.654260,-79.360636,9,2,28,65,2,1,0,3,6,12,0,3
16,Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306,6,2,27,65,2,0,0,6,6,12,1,3
24,"The Annex,North Midtown,Yorkville",M5R,Central Toronto,43.672710,-79.405678,10,0,32,65,4,0,0,3,4,12,0,3
30,Christie,M6G,Downtown Toronto,43.669542,-79.422564,7,0,23,72,2,2,0,1,3,12,0,3
32,"Little Portugal,Trinity",M6J,West Toronto,43.647927,-79.419750,6,0,26,78,1,0,0,3,2,10,0,3


In [57]:
toronto_final[toronto_final['Cluster'] == 1]


,Neighborhood,Postcode,Borough,Latitude,Longitude,Arts and Entertainment,Athletics and Sports,Educational,Food and Dine,Gym and Fitness Center,Health and Beauty Service,Hospitals,Others,Recreation,Stores and Utilities,Transportation,Cluster
12,Church and Wellesley,M4Y,Downtown Toronto,43.665860,-79.383160,7,0,50,65,4,2,6,5,6,11,0,1
14,"Ryerson,Garden District",M5B,Downtown Toronto,43.657162,-79.378937,4,1,50,63,2,0,6,4,5,21,0,1
15,St. James Town,M5C,Downtown Toronto,43.651494,-79.375418,6,0,50,65,3,0,2,5,4,17,0,1
17,Central Bay Street,M5G,Downtown Toronto,43.657952,-79.387383,7,1,51,63,3,0,6,4,4,17,0,1
18,"Adelaide,King,Richmond",M5H,Downtown Toronto,43.650571,-79.384568,11,0,51,62,4,2,6,4,1,14,1,1
20,"Design Exchange,Toronto Dominion Centre",M5K,Downtown Toronto,43.647177,-79.381576,8,2,50,67,3,1,3,5,7,6,1,1
21,"Commerce Court,Victoria Hotel",M5L,Downtown Toronto,43.648198,-79.379817,7,2,50,70,3,0,3,5,3,9,1,1
25,"Harbord,University of Toronto",M5S,Downtown Toronto,43.662696,-79.400049,4,0,50,70,1,0,6,5,2,16,0,1
26,"Chinatown,Grange Park,Kensington Market",M5T,Downtown Toronto,43.653206,-79.400049,7,1,47,71,3,0,5,5,0,13,0,1
28,Stn A PO Boxes 25 The Esplanade,M5W,Downtown Toronto,43.646435,-79.374846,6,2,49,69,3,0,3,5,2,12,1,1


In [58]:
toronto_final[toronto_final['Cluster'] == 4]


,Neighborhood,Postcode,Borough,Latitude,Longitude,Arts and Entertainment,Athletics and Sports,Educational,Food and Dine,Gym and Fitness Center,Health and Beauty Service,Hospitals,Others,Recreation,Stores and Utilities,Transportation,Cluster
4,Lawrence Park,M4N,Central Toronto,43.728020,-79.388790,0,0,11,3,1,0,1,1,1,1,0,4
10,Rosedale,M4W,Downtown Toronto,43.679563,-79.377529,0,1,10,12,0,0,1,2,3,4,1,4
22,Roselawn,M5N,Central Toronto,43.711695,-79.416936,0,1,14,13,2,1,0,0,0,6,0,4
27,"CN Tower,Bathurst Quay,Island airport,Harbourf...",M5V,Downtown Toronto,43.628947,-79.394420,1,1,4,4,0,0,0,1,6,0,2,4
38,Queen's Park,M9A,Downtown Toronto,43.667856,-79.532242,0,1,4,2,0,0,0,1,2,6,0,4


In [59]:
toronto_final[toronto_final['Cluster'] == 2]

,Neighborhood,Postcode,Borough,Latitude,Longitude,Arts and Entertainment,Athletics and Sports,Educational,Food and Dine,Gym and Fitness Center,Health and Beauty Service,Hospitals,Others,Recreation,Stores and Utilities,Transportation,Cluster
6,North Toronto West,M4R,Central Toronto,43.715383,-79.405678,0,6,16,25,2,2,0,0,4,4,0,2
8,"Moore Park,Summerhill East",M4T,Central Toronto,43.689574,-79.383160,1,0,16,38,4,0,0,4,6,8,0,2
11,"Cabbagetown,St. James Town",M4X,Downtown Toronto,43.667967,-79.367675,4,0,23,25,0,0,1,1,5,3,0,2
23,"Forest Hill North,Forest Hill West",M5P,Central Toronto,43.696948,-79.411307,0,1,17,26,4,1,0,2,4,6,0,2
31,"Dovercourt Village,Dufferin",M6H,West Toronto,43.669005,-79.442259,2,2,18,39,3,0,0,2,7,15,0,2
37,Business Reply Mail Processing Centre 969 Eastern,M7Y,East Toronto,43.662744,-79.321558,0,0,4,32,2,0,0,0,8,8,0,2


In [60]:
toronto_final[toronto_final['Cluster'] == 5]

,Neighborhood,Postcode,Borough,Latitude,Longitude,Arts and Entertainment,Athletics and Sports,Educational,Food and Dine,Gym and Fitness Center,Health and Beauty Service,Hospitals,Others,Recreation,Stores and Utilities,Transportation,Cluster
0,The Beaches,M4E,East Toronto,43.676357,-79.293031,1,1,18,55,0,3,0,3,9,12,0,5
2,"The Beaches West,India Bazaar",M4L,East Toronto,43.668999,-79.315572,4,1,7,49,3,1,0,2,11,10,1,5
9,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",M4V,Central Toronto,43.686412,-79.400049,1,1,15,49,5,2,0,2,4,9,3,5
19,"Harbourfront East,Toronto Islands,Union Station",M5J,Downtown Toronto,43.640816,-79.381752,10,6,14,60,2,1,1,3,12,5,1,5
33,"Brockton,Exhibition Place,Parkdale Village",M6K,West Toronto,43.636847,-79.428191,5,3,13,64,4,1,0,5,2,16,0,5
34,"High Park,The Junction South",M6P,West Toronto,43.661608,-79.464763,2,1,16,62,3,2,0,3,3,20,2,5
35,"Parkdale,Roncesvalles",M6R,West Toronto,43.648960,-79.456325,5,1,11,56,1,0,1,7,8,21,1,5
36,"Runnymede,Swansea",M6S,West Toronto,43.651571,-79.484450,2,1,15,54,2,1,0,1,3,9,1,5
